In [0]:
from pyspark.sql import functions as F
events = spark.read.csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Oct.csv", header=True, inferSchema=True)

In [0]:
events.describe(["price"]).show()

In [0]:
from pyspark.sql.functions import to_date, col


events = events.withColumn(
    "event_date",
    to_date(col("event_time"))
)

In [0]:
# Hypothesis: weekday vs weekend conversion
weekday = events.withColumn("is_weekend",
    F.dayofweek("event_date").isin([1,7]))
weekday.groupBy("is_weekend", "event_type").count().show()

In [0]:
events.createOrReplaceTempView("oct_events")

In [0]:
%sql 
MERGE INTO default.gold_product_performance AS g
USING (
    SELECT
        product_id,
        AVG(price) AS price
    FROM oct_events
    WHERE price IS NOT NULL
    GROUP BY product_id
) AS s
ON g.product_id = s.product_id

WHEN MATCHED THEN
  UPDATE SET g.price = s.price;

In [0]:
events2 = spark.read.table("default.gold_product_performance")

In [0]:
events2.stat.corr("price", "conversion_rate")

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
w = Window.partitionBy("user_id")
features = (
    events
    .withColumn("hour", F.hour("event_time"))
    .withColumn("day_of_week", F.dayofweek("event_time"))
    .withColumn("is_weekend", F.dayofweek("event_time").isin([1, 7]).cast("int"))
    .withColumn("price_log", F.log1p("price"))
    .withColumn("first_event_time", F.min("event_time").over(w))
    .withColumn(
        "time_since_first_event",
        F.unix_timestamp("event_time") - F.unix_timestamp("first_event_time"))
)
features.select(
    "user_id",
    "event_type",
    "hour",
    "day_of_week",
    "is_weekend",
    "price_log",
    "time_since_first_event"
).show(5, truncate=False)